In [ ]:
!pip install kaggle

In [ ]:
!pip install statsmodels

In [ ]:
pip install pmdarima


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.graphics.tsaplots import acf
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.arima_model import ARMA
from statsmodels.tsa.seasonal import seasonal_decompose
import pmdarima as pm

In [ ]:
!kaggle datasets download -d sumanthvrao/daily-climate-time-series-data

In [ ]:
import zipfile

In [ ]:
# we have more than one csv files in the zip file and I need to load the two of them
zip_file_path = 'daily-climate-time-series-data.zip'

# Step 1: Read both CSV files directly from the ZIP archive
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    with zip_ref.open('DailyDelhiClimateTrain.csv') as train_file:
        df_train = pd.read_csv(train_file)

    with zip_ref.open('DailyDelhiClimateTest.csv') as test_file:
        df_test = pd.read_csv(test_file)

Let have a lok at the first few rows of our datasets


In [ ]:
df_train.head(n=9)

In [ ]:
df_test.head(n=7)

Let now see the shapes of these datasets

In [ ]:
df_train.shape

In [ ]:
df_test.shape

Now lets see the data types of the columns

In [ ]:
df_train.dtypes

Here we need to convert the 'date' column to a datetime object and make it to be the index column.

In [ ]:
df_train['date'] = pd.to_datetime(df_train['date'], format = '%Y-%m-%d')


In [ ]:
df_train = df_train.set_index('date')


The frequency of the this dataset is not set, I will proceed and set it to 'D' for days.

In [ ]:
df_train = df_train.asfreq('D')

I will repeat the same process with the test dataset.

In [ ]:
df_test.dtypes

In [ ]:
df_test['date'] = pd.to_datetime(df_test['date'], format = '%Y-%m-%d')


In [ ]:
df_test = df_test.set_index('date')


In [ ]:
df_test = df_test.asfreq('D')

We can now start our analysis

In [ ]:
df_train.isna().sum()

In [ ]:
df_train.describe()


In [ ]:
print('correlation coefficients among the four aspects')
print('meantemp and humidity correlation: \t\t' + str(df_train['meantemp'].corr(df_train['humidity'])))
print('meantemp and wind_speed correlation: \t\t' + str(df_train['meantemp'].corr(df_train['wind_speed'])))
print('humidity and wind_speed correlation: \t\t' + str(df_train['humidity'].corr(df_train['wind_speed'])))
print('meantemp and meanpressure correlation: \t\t' + str(df_train['meantemp'].corr(df_train['meanpressure'])))
print('humidity and meanpressure correlation: \t\t' + str(df_train['humidity'].corr(df_train['meanpressure'])))
print('wind_speed and meanpressure correlation: \t' + str(df_train['wind_speed'].corr(df_train['meanpressure'])))


Meanpressure is weakly correlated with the pther three.

In [ ]:
sns.clustermap(df_train.corr(method='pearson'), figsize=(6,4))
plt.title('Correlation Analysis')

For this reason I will not consider meanpressure in this analysis.

In [ ]:
del(df_train['meanpressure'])

In [ ]:
df_train.plot.area(figsize = (8, 5))
plt.xlabel('Time')
plt.title('All Changes')
plt.show


In [ ]:
col_names = df_train.columns
for col in col_names:
     plt.figure(figsize=(12, 5))
     # Defining subplots in one row and two columns
     # In index one of the subplot, we will have histogram
     plt.subplot(1, 3, 1)
     sns.histplot(data=df_train, x=col, kde=True, color = '#911116', line_kws={'color': '#911156'})
     plt.xlabel(col)
     plt.ylabel('Frequency')
     plt.title(f'Distribution of {col}')
     # The second index subplot will have boxplots
     plt.subplot(1, 3, 2)
     sns.boxplot(data=df_train, y=col)
     plt.ylabel(col)
     plt.title(f'Box Plot of {col}')
     # The third plot will display the observations
     plt.subplot(1,3,3)
     sns.lineplot(data=df_train, x=df_train.index, y=col)
     plt.xlabel('Time')
     plt.ylabel(col)
     plt.title(f'Observations of {col}')
     plt.xticks(rotation=90)
     # To make sure that no overlapping 
     plt.tight_layout()
     plt.show()

Lets observe the monthly distributions

In [ ]:
monthly = df_train.groupby(df_train.index.month).mean()
plt.figure(figsize=(12,5))
sns.lineplot(data=monthly)
plt.xlabel('Month')
plt.title('Monthly Observations ')
plt.xticks(rotation=90)
plt.legend()
plt.show()

The seasonality here is quite clear.

Lets now decompose this dataset into trend, seasonality and noise.

In [ ]:
for col in col_names:
    decomposition = seasonal_decompose(df_train[col], period=365)
    decomposition.plot()
    plt.show()

We see a strong seasonality in Meantemp, Humidity, Windspeed.

Also, Meantemp has an upward trend over the year. 

Lets now observe the Argumented Dicke-Fuller test and see the p-values.


In [ ]:
for col in col_names:
    print('The p-value for ' + col + ' is: ' + str(adfuller(df_train[col])[1]))


Meantemp is non-stationary, remember that the Argumented Dickey-Fuller test tests only the trend. We can not see the seasonality trend here but we know it is present in all the three aspects. 

Now we will see the the ACF and PACF plots 

Remember the frequency of our train data is set to daily.

In [ ]:
for col in col_names:
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 3))
    
    # The first index subplot will have ACF
    plot_acf(df_train[col], lags=60, zero=False, ax=ax1)
    ax1.set_title(f'ACF of {col}')
    
    # The second index subplot will have PACF
    plot_pacf(df_train[col], lags=60, zero=False, ax=ax2)
    ax2.set_title(f'PACF of {col}')
    
    # To make sure that no overlapping 
    plt.tight_layout()
    plt.show()

You can see non-stationarity in our data.

Before fitting a model, we need to know the seasonal and non-seasonal differencing orders. We will start by detrending the non-seasonal trend for meantemp and check the adfuller p-value after the first differencing. 

In [ ]:
meantemp_diff = df_train['meantemp'].diff().diff(365)
meantemp_diff = meantemp_diff.dropna()
decomp = seasonal_decompose(meantemp_diff, period =365)
decomp.plot()
plt.title('First seasonal and non-seasonal difference')
plt.show()
print('The p-value for  meantemp_diff is: ' + str(adfuller(meantemp_diff)[1]))


In [ ]:
decomp.seasonal.plot()
plt.title('seasonal plot')
plt.show()


The meantemp is stationary after one differencing of both seasonal and non-seasonal trends.

You can see that the seasonal and non-seasonal trends are now detrended, all by the first difference order.

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 3))
    
plot_acf(meantemp_diff, lags=60, zero=False, ax=ax1)
ax1.set_title('ACF for Stationary meantemp')
    
plot_pacf(meantemp_diff, lags=60, zero=False, ax=ax2)
ax2.set_title('PACF for Stationary meantemp')

plt.show()

Lets now fit a SARIMA model.

In [ ]:
df_train.columns


In [ ]:
!pip install --upgrade statsmodels

In [ ]:
from statsmodels.tsa.statespace.sarimax import SARIMAX


In [ ]:
model = SARIMAX(df_train['meantemp'], order=(4, 1, 3), seasonal_order=(4, 1, 3, 365))
result = model.fit()
result.summary()

In [ ]:
# Extract the target variable and exogenous variables
# y = df_train['meantemp']
# X = df_train[['humidity', 'wind_speed']]  # Pass exogenous variables as a DataFrame

# Fit the ARIMA model with exogenous variables
# results = pm.auto_arima(y, exogenous=X, seasonal=True, m=365, max_d=1, max_D=1, trend='ct',
                        # stepwise=False, suppress_warnings=True, error_action='ignore',
                        # max_p=6, max_q=6, max_Q=4, max_P=4, n_jobs=-1, information_criterion='aic')
                        
